### 基于概率论的分类方法：朴素贝叶斯

KNN和Decision Tree是要求分类器做出艰难决策，即给出某一个数据实例属于哪一类这类问题的确定答案。但分类器有时会有错误，此时可以要求分类器给出一个最优的类别猜测结果，同时给出这个猜测的概率估计值。

#### 4.1 基于贝叶斯决策理论的分类方法
##### 朴素贝叶斯
- 优点：在数据较少的情况下仍然有效，可以处理多类别问题
- 缺点：对于输入数据的准备方式较为敏感
- 适用数据类型：标称型数据
##### 贝叶斯理论
假设有一数据集，两类数据构成，p1(x,y）表示数据点（x,y)属于第一类，p2(x,y)表示属于第二类。
对于一个新数据点(x,y)，则可以用如下规则判断其类别：

- if p1(x,y)>p2(x,y), then (x,y) -> Type1
- if p1(x,y)<p2(x,y), then (x,y) -> Type2

选择最高概率的决策，就是贝叶斯决策理论的思想 

#### 条件概率
$p(c|x)=\frac{p(x|c)p(c)}{p(x)}$



##### 朴素贝叶斯算法朴素的原因是：
1. 假设各个特征之间相互独立
2. 假设各个特征重要程度一致。